In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 46.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b64f44a09ad7f2447317244d840a3e61bf69fc655fbcafac8698a0819c572dc4
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [36]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()

--2021-04-12 15:31:08--  https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
Resolving www-us.apache.org (www-us.apache.org)... 207.244.88.140
Connecting to www-us.apache.org (www-us.apache.org)|207.244.88.140|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz [following]
--2021-04-12 15:31:09--  https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224374704 (214M) [application/x-gzip]
Saving to: ‘spark-3.1.1-bin-hadoop2.7.tgz’

spark-3.1.1-bin-had 100%[===================>] 213.98M  17.9MB/s    in 16s     

2021-04-12 15:31:26 (13.3 MB/s) - ‘spark-3.1.1-bin-hadoop2.7.tgz’ saved [224374704/224374704]

In [ ]:
!gdown --id 1zAhxB-lqewSdqusvatGE0BE9Goms8_lp

!gdown --id 1UB4iA9CFdAAR8cGKDvehUT6j8Gj38b-1
!gdown --id 1hTeTMFufhILndQAFLntTM9c25RF-fyBm
!gdown --id 1zq7UjYgC-CMsYODF58OYmMBEcshqdyaF

Downloading...
From: https://drive.google.com/uc?id=1zAhxB-lqewSdqusvatGE0BE9Goms8_lp
To: /content/data_scaled_means.csv
100% 10.9k/10.9k [00:00<00:00, 25.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UB4iA9CFdAAR8cGKDvehUT6j8Gj38b-1
To: /content/EllipticEnvelope
100% 116k/116k [00:00<00:00, 1.02MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hTeTMFufhILndQAFLntTM9c25RF-fyBm
To: /content/IsolationForest
100% 156k/156k [00:00<00:00, 1.38MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zq7UjYgC-CMsYODF58OYmMBEcshqdyaF
To: /content/LocalOutlierFactor
100% 10.0k/10.0k [00:00<00:00, 19.2MB/s]


In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [111]:
import pickle
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession \
    .builder \
    .appName("Spark Dataframe") \
    .config("spark.some.config.option") \
    .getOrCreate()

In [68]:
data_scaled_means = spark.read.csv("data_scaled_means.csv",header=True, sep=",", inferSchema =True)
data_scaled_means.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+-------------+----------+-----------+------------+---------------------------+-----------------------------------+---------------------+---------------------------------------+-----------------------------------------------+--------------------------------------+----------------------------+-----------------+---------------------+-----------------+----------------------------------------------------------+----------------+------------+---------------------------+--------------------------+---------------+-------------+-------------+-

In [254]:
@udf(returnType=DoubleType())
def predictor(*kwargs):
    #open picked model
    model = open("IsolationForest", "rb")
    ilf_model = pickle.load(model)
    model.close()
    print(kwargs)
    answerIF_proba = abs(ilf_model.score_samples([kwargs]))

 
    model = open("LocalOutlierFactor", "rb")
    lof_model = pickle.load(model)
    model.close()
    answerLOF_proba = lof_model.decision_function([kwargs])
    answerLOF_proba = 1 - ((answerLOF_proba + 0.9118517621467248) / (0.5391638200654012 + 0.9118517621467248)) 

    model = open("EllipticEnvelope", "rb")
    ee_model = pickle.load(model)
    model.close()
    answerEE_proba = ee_model.decision_function([kwargs])
    answerEE_proba = 1 - (answerEE_proba - 3 * -1.6697754290362354e-13) * 10 ** 12

    return (float(answerIF_proba*2) + float(answerLOF_proba*1) + float(answerEE_proba*2)) / 5

In [255]:
column_input = []
for c in data_scaled_means.columns:
  column_input.append(data_scaled_means[c])

df_prediction = data_scaled_means.withColumn("prediction",
                                            predictor(*column_input))


In [263]:
result = [list(row)[0] for row in df_prediction.select('prediction').collect()]
result

[0.33536269161026055,
 0.4600479559377632,
 0.37434271087540216,
 0.27554091202299225,
 0.3684540532999103,
 0.4660563016589777,
 0.30256782671783367,
 0.3052450340189892,
 0.37446040063245845,
 0.34654855821089336,
 0.33263167275641425,
 0.31682058688101217,
 0.3435323760248429,
 0.5063452958437309,
 0.489596162861144,
 0.4107104555137072]